In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import os
import numpy as np
#from sklearn.mixture import GaussianMixture
import matplotlib.ticker as plticker
import pybedtools
import os
import os.path
import pandas as pd


## Classify samples as having ecDNA-chromothripsis (the ecDNA and chromothripsis samples overlap) or ecDNA, no chromothripsis (the events are distinct do not overlap) ##

In [ ]:
ns = pd.read_table("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/ecDNA-chromothripsis-summary_nonsmoker_intersection.tsv")
s = pd.read_table("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/ecDNA-chromothripsis-summary_smoker_intersection.tsv")

#stack dataframes on top of each other
aa_merged = pd.concat([ns, s], axis=0)

barcode = []
aa_merged = pd.read_table("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/ecDNA-chromothripsis-summary_all_intersection.tsv", sep="\t")
for s in aa_merged['ID']:
    barcode.append(s.split('_')[0])

aa_merged['Sample'] = barcode

aa_merged['Annotation'] = aa_merged['Annotation'].astype('category')
aa_merged['Annotation'] = aa_merged['Annotation'].cat.set_categories(["ecDNA, chromothripsis", "ecDNA, no chromothripsis"], ordered=True)
aa_merged.sort_values(by=["Sample", 'Annotation'], inplace=True, ascending=True)
aa_merged = aa_merged.drop_duplicates(subset=["Sample"]) #now we are at the sample level
aa_merged.head()


#NOW WE"RE AT THE SAMPLE LEVEL
result = aa_merged[["Sample", "overlap", "Annotation", "length_chromothripsis", "length_ecDNA"]]

#replace NaN with 0
result = result.fillna(0)
#output to tsv
result.to_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/ecDNA-chromothripsis.txt", sep="\t", index=False)


In [ ]:
result.info()

## Obtain set of overlapping samples #

In [ ]:
data = pd.read_table("~/iCloud/dev/Sherlock-Lung/results2/log-reg-table5.tsv", sep="\t")
#Select rows Smoking=="Smoker" & Histology=="Adenocarcinoma"

#data = data[(data["Smoking"]=="Smoker")]

chromo = data[data["Chromothripsis"] == 1]["Tumor_Barcode"]
ecDNA = data[data["ecDNA_status"] == 1]["Tumor_Barcode"]

intersection = set(chromo) & set(ecDNA)
len(intersection)


## Intersect ecDNA regions with chromothripsis regions and compute overlap ##

In [ ]:
ecDNA_dir = "/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/rescaled/ecDNA"
sv_dir = "/Users/khandekara2/iCloud/dev/Sherlock-Lung/data/ShatterSeek-input/output/"
genome_file = "/Users/khandekara2/iCloud/dev/Sherlock-Lung/data/ShatterSeek-input/SV/GRCh38.genome"

metadata = pd.read_csv("/Users/khandekara2/iCloud/dev/Sherlock-Lung/metadata/wgs_data_full_all_1217_final.txt", sep="\t")
mapping = dict(zip(metadata["Tumor_Sample_ID"], metadata["Tumor_Barcode"]))

os.chdir("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/rescaled/ecDNA")

# Convert the chromosome part to numeric while keeping 'X', 'Y', and 'MT' intact (if present)
# Sorting chromosomes with numbers first, followed by 'X', 'Y', and 'MT'
def custom_sort_key(chrom):
    try:
        # Return numeric for chromosomes like chr1, chr2, etc.
        return int(chrom)
    except ValueError:
        # Handle special cases like chrX, chrY, and chrM (or chrMT)
        if chrom == 'X':
            return 23
        elif chrom == 'Y':
            return 24
        elif chrom == 'M' or chrom == 'MT':
            return 25
        else:
            return 100  # Default for any other potential chromosomes

for file in os.listdir("."):
    
    ecDNA = pd.read_csv(file, sep="\t", header=None)
    sample = file.split(".")[0]
    
    #if file exists in chromothripsis directory
    #check if file exists
    if sample not in mapping or not os.path.exists("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/" + mapping[sample] + "_chromothripsis.tsv"):
        print(sample)
        continue
        # os.chdir("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/")
        # for f in os.listdir("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/"):
        #     if sample in f:
        #         sv = pd.read_csv(f, sep="\t")
        #         if sv.shape[1] >= 1:
        #             sv["chrom"] = sv["chrom"].astype(int)
        #             sv["start"] = sv["start"].astype(int)
        #             sv["end"] = sv["end"].astype(int)
        #             sv = sv.sort_values(by=['chrom', 'start'], key=lambda x: x.map(custom_sort_key))
        #             sv[sv.columns[0]] = "chr" + sv[sv.columns[0]].astype(str)
                    
        #             chromo_file = "/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/" + sample + "_chromothripsis.tsv"
        #             a = pybedtools.BedTool(file) #ecDNA
        #             b = pybedtools.BedTool.from_dataframe(sv) #chromothripsis region

        #             #wao- Write the original A and B entries plus the number of base pairs of overlap between the two features.
        #             #However, A features w/o overlap are also reported with a NULL B feature and overlap = 0.
        #             c = a.intersect(b, wao=True, g=genome_file)

    else:
        print("Found: " + "/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/" + mapping[sample] + "_chromothripsis.tsv")
        sv = pd.read_csv("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/" + mapping[sample] + "_chromothripsis.tsv", sep="\t")
        if sv.shape[1] >= 1:
            sv["chrom"] = sv["chrom"].astype(int)
            sv["start"] = sv["start"].astype(int)
            sv["end"] = sv["end"].astype(int)
            sv = sv.sort_values(by=['chrom', 'start'], key=lambda x: x.map(custom_sort_key))
            sv[sv.columns[0]] = "chr" + sv[sv.columns[0]].astype(str)
            
            chromo_file = "/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/" + sample + "_chromothripsis.tsv"
            a = pybedtools.BedTool(file) #ecDNA
            b = pybedtools.BedTool.from_dataframe(sv) #chromothripsis region


            #wao- Write the original A and B entries plus the number of base pairs of overlap between the two features.
            #However, A features w/o overlap are also reported with a NULL B feature and overlap = 0.
            c = a.intersect(b, wao=True, g=genome_file)

            result_df = c.to_dataframe()
            result_df.columns = final.columns.tolist() + ["chrom1", "start1", "stop1", "Sample", "ID", "chrom2", "start2", "stop2", "Tumor_Barcode", "overlap"]
            result_df.to_csv("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/" + mapping[sample] + ".bed", sep="\t", index=False)
            #c.saveas("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/rescaled/ecDNA_chromothripsis/" + sample + "_ecDNA_chromothripsis.tsv")
            print("Saved: " + "/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/" + mapping[sample] + ".bed")



In [ ]:
result_df.head()

## LOOP THROUGH CHROMOTHRIPSIS AND ECDNA INTERSECTION FILES. EACH LINE IS A GENOMIC SEGMENT IN AN ECDNA. ##

In [ ]:
l = []
os.chdir("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis")
for file in os.listdir("."):
    if file.endswith("_ecDNA-chromothripsis.bed"):
        sample = file.split("_")[0]
        if sample in intersection:
            print(sample)
            df = pd.read_csv(file, sep="\t")
            assert(len(df.columns) == 11)
            l.append(df)

master = pd.concat(l)

#fix the annotation here
#change annotation column to chromothripsis, no ecDNA if overlap == 0 and chromothripsis, ecDNA if overlap > 0
master["Annotation"] = np.where(master["overlap"] == 0, "ecDNA, no chromothripsis", "ecDNA, chromothripsis")
    

In [ ]:
master

In [ ]:
master.info()

In [ ]:
#convert columns to type int
master["start1"] = master["start1"].astype(int)
master["stop1"] = master["stop1"].astype(int)
# master["start2"] = master["start2"].astype(int)
# master["stop2"] = master["stop2"].astype(int)

In [ ]:
list(master["start1"])

In [ ]:
s = master[master["start1"] == "start"]

In [ ]:
s.head()

In [ ]:
#d = master[master["overlap"] == 0]
d.head()

In [ ]:
master.head()

In [ ]:
length_ecDNA = master["stop1"] - master["start1"]
length_chromothripsis = master["stop2"] - master["start2"]
# Use GroupBy() & compute sum on specific column
df2 = master.groupby('ID')['length_ecDNA'].sum()
#print("Get sum of the grouped data:\n", df2)


## LOOP THROUGH CHROMOTHRIPSIS FILES AND FIND ALL THE SAMPLES WITH CHROMOTHRIPSIS THAT DID NOT HAVE ECDNA AND WRITE OUT THAT TEN COLUMN TABLE ##

In [ ]:
#NOW 
os.chdir("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/")
ecDNA_path = "/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/ecDNA-regions/rescaled/ecDNA/"
all_dfs = []
for file in os.listdir("."):
    if file.endswith("_chromothripsis.tsv"):
        sample = file.split("_")[0]
        if not os.path.isfile(ecDNA_path + sample + ".bed"):
            if (sample in mapping) and (not os.path.isfile(ecDNA_path + mapping[sample] + ".bed")): #chromothripsis without ecDNA
                chromo_df = pd.read_csv(file, sep="\t")
                ecDNA_df = pd.DataFrame(np.nan, index=range(chromo_df.shape[0]), columns=["chrom1", "start1", "stop1", "Sample", "ID"])
                result_df = pd.concat([ecDNA_df, chromo_df], axis=1)
                result_df["overlap"] = [0 for x in range(result_df.shape[0])]
                result_df.columns = ["chrom1", "start1", "stop1", "Sample", "ID", "chrom2", "start2", "stop2", "Tumor_Barcode", "overlap"]
                result_df["Annotation"] = ["chromothripsis, no ecDNA" for x in range(result_df.shape[0])]
                all_dfs.append(result_df)

df = pd.concat(all_dfs)
df.head()

In [ ]:
df.info()

In [ ]:
master = pd.read_csv("/Users/khandekara2/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/ecDNA-chromothripsis-master-table.tsv", sep="\t")
master.head()

In [ ]:
#change the NaN values in start1 and stop1 to 0
master["start1"] = master["start1"].fillna(0)
master["stop1"] = master["stop1"].fillna(0)

In [ ]:
master.Sample.nunique()

In [ ]:
#randomly generate a string ID and replace the NAs in the ID column
import random
import string
def generate_random_id(length=10):
    """Generate a random string ID of specified length."""
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

# Assuming df is your dataframe:
def fill_na_ids(df):
    """Fill NA values in ID column with random strings."""
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Find NA locations
    na_mask = df['ID'].isna()
    
    # Generate unique random IDs for each NA
    na_count = na_mask.sum()
    if na_count > 0:
        print(f"Filling {na_count} NA values with random IDs")
        random_ids = [generate_random_id() for _ in range(na_count)]
        
        # Fill NA values
        df.loc[na_mask, 'ID'] = random_ids
    
    return df

# Example usage:
master = fill_na_ids(master)


In [ ]:
master.isnull().sum(axis = 0)

In [ ]:
master.Annotation.value_counts()

In [ ]:
c = master[master["Annotation"] == "chromothripsis, no ecDNA"]
c.head()

## Here we perform a groupby operation to go from individual segments to ecDNA-level (each row represents an ecDNA). ##

In [ ]:
length_ecDNA = master["stop1"] - master["start1"]
length_chromothripsis = master["stop2"] - master["start2"]
master["length_ecDNA"] = length_ecDNA
master["length_chromothripsis"] = length_chromothripsis

#get all rows where overlap is 
#master2 = 

result = master.groupby('ID').agg({
    'length_ecDNA': 'sum',
    'overlap': 'sum',
    'length_chromothripsis': 'median',
    "Annotation": lambda x: x.mode().iloc[0]
}).reset_index()

In [ ]:
result.Annotation.value_counts(dropna=False)

In [ ]:
result

In [ ]:
# if overlap is 0 and length chromothripsis is NaN, them multiply length ecDNA by -1
result.loc[(result["overlap"] == 0) & (result["length_chromothripsis"].isnull()), "length_ecDNA"] *= -1

In [ ]:
#if the length chromothripsis is 0, change the Annotation to ecDNA, no chromothripsis
result.loc[result["length_chromothripsis"] == 0, "Annotation"] = "ecDNA, no chromothripsis"


In [ ]:
#output to csv
#remove the string stem "_interval" from the ID column
#result["ID"] = result["ID"].str.replace("_interval", "")
result.to_csv("/Users/azhark/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/ecDNA-chromothripsis-summary_all_intersection.tsv", sep="\t", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#remove the string stem "_interval" from the ID

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def create_genomic_barplot(data, bar_height=0.4):
    """
    Create a horizontal bar plot showing ecDNA and chromothripsis regions.
    """
    # Clean the data
    data = data.copy()
    
    # Convert lengths from base pairs to Mb
    for col in ['length_ecDNA', 'overlap', 'length_chromothripsis']:
        data[col] = data[col].fillna(0) / 1e6
    
    # Create categories
    def get_category(row):
        if row['overlap'] > 0:
            return 1  # overlap category (highest priority)
        elif row['length_ecDNA'] > 0:
            return 2  # chromothripsis only
        else:
            return 3  # ecDNA only
            
    data['category'] = data.apply(get_category, axis=1)
    
    # Within each category, sort by the appropriate size:
    # - For overlap category: sort by chromothripsis size
    # - For chromothripsis only: sort by chromothripsis size
    # - For ecDNA only: sort by ecDNA size
    def get_sort_size(row):
        if row['category'] in [1, 2]:  # overlap or chromothripsis only
            return row['length_chromothripsis']
        else:  # ecDNA only
            return abs(row['length_ecDNA'])
            
    data['sort_size'] = data.apply(get_sort_size, axis=1)
    
    # Sort first by category, then by the appropriate size within each category
    data = data.sort_values(['category', 'sort_size'], 
                          ascending=[True, False])
    
    # Calculate figure size
    num_samples = len(data)
    fig_height = num_samples * 0.25  # 0.25 inch per sample
    figsize = (15, fig_height)
    
    # Set up clean style manually
    plt.style.use('default')
    plt.rcParams['axes.grid'] = True
    plt.rcParams['grid.alpha'] = 0.3
    plt.rcParams['grid.color'] = '#cccccc'
    
    # Create figure with exact dimensions needed
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0.1, 0.1, 0.7, 0.85])
    
    # Calculate positions for bars
    positions = range(len(data))
    
    # Plot chromothripsis regions (light blue)
    chromothripsis_mask = data['length_chromothripsis'] > 0
    if chromothripsis_mask.any():
        ax.barh(
            y=[pos for pos, mask in zip(positions, chromothripsis_mask) if mask],
            width=data[chromothripsis_mask]['length_chromothripsis'],
            height=bar_height,
            label='Chromothripsis',
            color='#87CEEB',
            alpha=0.9
        )
    
    # Plot overlapping regions (red)
    overlap_mask = data['overlap'] > 0
    if overlap_mask.any():
        ax.barh(
            y=[pos for pos, mask in zip(positions, overlap_mask) if mask],
            width=data[overlap_mask]['overlap'],
            height=bar_height,
            label='ecDNA + Chromothripsis overlap',
            color='#FF6B6B',
            alpha=0.9
        )
    
    # Plot ecDNA-only regions (purple, negative direction)
    ecdna_only_mask = data['length_ecDNA'] < 0
    if ecdna_only_mask.any():
        ax.barh(
            y=[pos for pos, mask in zip(positions, ecdna_only_mask) if mask],
            width=abs(data[ecdna_only_mask]['length_ecDNA']),
            height=bar_height,
            label='ecDNA without Chromothripsis',
            color='#6A4C93',
            alpha=0.9,
            left=-abs(data[ecdna_only_mask]['length_ecDNA'])
        )
    
    # Plot ecDNA regions with chromothripsis but no overlap (light blue)
    ecdna_chromo_no_overlap = (data['length_ecDNA'] > 0) & (data['overlap'] == 0)
    if ecdna_chromo_no_overlap.any():
        ax.barh(
            y=[pos for pos, mask in zip(positions, ecdna_chromo_no_overlap) if mask],
            width=data[ecdna_chromo_no_overlap]['length_ecDNA'],
            height=bar_height,
            label='ecDNA (no overlap)',
            color='#87CEEB',
            alpha=0.9
        )
    
    # Customize plot
    ax.set_yticks(positions)
    ax.set_yticklabels(data['ID'], fontsize=6)
    
    # Set y-axis limits explicitly
    ax.set_ylim(len(data) - 0.5, -0.5)
    
    # Add vertical line at x=0
    ax.axvline(x=0, color='black', linewidth=0.5)
    
    # Set x-axis limits to be symmetric
    max_abs_x = max(
        abs(data['length_ecDNA'].min()),
        abs(data['length_chromothripsis'].max())
    )
    ax.set_xlim(-max_abs_x * 1.1, max_abs_x * 1.1)
    
    # Customize layout
    ax.set_xlabel('Size of region (Mb)')
    ax.set_title('ecDNA and Chromothripsis Regions by Sample', pad=20)
    
    # Add legend with adjusted position
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Add summary statistics
    stats_text = (
        f"Total samples: {len(data)}\n"
        f"Samples with overlap: {(data['category'] == 1).sum()}\n"
        f"Chromothripsis only: {(data['category'] == 2).sum()}\n"
        f"ecDNA only: {(data['category'] == 3).sum()}\n"
        f"Average ecDNA size: {abs(data['length_ecDNA']).mean():.1f} Mb"
    )
    
    plt.figtext(0.98, 0.5, stats_text, 
                bbox=dict(facecolor='white', alpha=0.8),
                verticalalignment='center')
    
    return fig, ax

# Example usage with your data
# Assuming the data is in a string format, we'll use StringIO to read it
from io import StringIO


# Read and process the data
#df = pd.read_csv('your_data.csv', sep='\t')

# Create plot
fig, ax = create_genomic_barplot(result)
plt.show()


In [ ]:
#out

In [ ]:
df = result
# Convert lengths to Mb and look at value distributions
print("Number of total rows:", len(df))
print("\nNumber of rows with non-zero values:")
print("length_ecDNA:", (df['length_ecDNA'] != 0).sum())
print("overlap:", (df['overlap'] != 0).sum())
print("length_chromothripsis:", (df['length_chromothripsis'].notna()).sum())

# Look at the last 10 rows of the data to see what's there
print("\nLast 10 rows of the data:")
print(df.tail(10)[['ID', 'length_ecDNA', 'overlap', 'length_chromothripsis', 'Annotation']])

# Check for any NaN or null values
print("\nNumber of NaN values in each column:")
print(df.isna().sum())


In [ ]:
def create_genomic_barplot(data, bar_width=0.8):  # Made bars wider
    """
    Create a vertical bar plot showing ecDNA and chromothripsis regions.
    """
    # Clean the data
    data = data.copy()
    
    # Convert lengths from base pairs to Mb
    for col in ['length_ecDNA', 'overlap', 'length_chromothripsis']:
        data[col] = data[col].fillna(0) / 1e6
    
    # Fix the categories based on overlap
    data['category'] = data['Annotation']
    data.loc[data['overlap'] > 0, 'category'] = 'ecDNA, chromothripsis'
    
    # Create sort order
    def get_sort_values(row):
        if row['overlap'] > 0:
            return (1, row['length_chromothripsis'])
        elif row['category'] == 'ecDNA, chromothripsis':
            return (2, row['length_chromothripsis'])
        elif row['category'] == 'chromothripsis, no ecDNA':
            return (3, row['length_chromothripsis'])
        else:  # ecDNA, no chromothripsis
            return (4, abs(row['length_ecDNA']))
    
    # Apply sorting
    data['sort_1'] = data.apply(lambda x: get_sort_values(x)[0], axis=1)
    data['sort_2'] = data.apply(lambda x: get_sort_values(x)[1], axis=1)
    data = data.sort_values(['sort_1', 'sort_2'], ascending=[True, False])
    
    # Calculate figure size based on number of samples
    num_samples = len(data)
    
    # Set up clean style manually
    plt.style.use('default')
    plt.rcParams['axes.grid'] = True
    plt.rcParams['grid.alpha'] = 0.3
    plt.rcParams['grid.color'] = '#cccccc'

    # Adjust font sizes for publication
    plt.rcParams['font.size'] = 8  # base font size
    plt.rcParams['axes.labelsize'] = 8
    plt.rcParams['axes.titlesize'] = 9
    plt.rcParams['legend.fontsize'] = 8 
    
    # Publication-quality figure size (double column)
    fig = plt.figure(figsize=(7.2, 4), dpi=300)  # width=7.2 inches (183 mm), height adjusted for good aspect ratio
    ax = fig.add_axes([0.1, 0.15, 0.85, 0.75])  # Add this line here


    # Calculate positions for bars
    positions = range(len(data))
    
    # Plot bars
    for idx, row in data.iterrows():
        pos = data.index.get_loc(idx)
        
        if row['overlap'] > 0:
            ax.bar(pos, row['length_chromothripsis'], width=bar_width,
                  color='#87CEEB', alpha=0.9)
            ax.bar(pos, row['overlap'], width=bar_width,
                  color='#FF6B6B', alpha=0.9)
            
        elif row['category'] == 'ecDNA, chromothripsis':
            ax.bar(pos, row['length_chromothripsis'], width=bar_width,
                  color='#87CEEB', alpha=0.9)
            
        elif row['category'] == 'chromothripsis, no ecDNA':
            ax.bar(pos, row['length_chromothripsis'], width=bar_width,
                  color='#87CEEB', alpha=0.9)
            
        elif row['category'] == 'ecDNA, no chromothripsis':
            ax.bar(pos, -abs(row['length_ecDNA']), width=bar_width,
                  color='#6A4C93', alpha=0.9)
    
    # Customize plot
    ax.set_xticks(positions)
    #ax.set_xticklabels(data['ID'], fontsize=8, rotation=45, ha='right')  # Increased font size, adjusted rotation

    ax.set_xticks([])  # removes tick marks
    ax.spines['bottom'].set_visible(False)  # removes the bottom line of the plot
    
    # Set x-axis limits with some padding
    ax.set_xlim(-0.5, len(data) - 0.5)
    
    # Add horizontal line at y=0
    ax.axhline(y=0, color='black', linewidth=0.5)
    
    min_y = data['length_ecDNA'].min()  # actual minimum value
    max_y = data['length_chromothripsis'].max()  # actual maximum value
    ax.set_ylim(min_y * 1.1, max_y * 1.1)  # just add 10% padding on each end
    
    # Customize layout with larger fonts
    ax.set_ylabel('Size of region (Mb)', fontsize=12)
    ax.set_title('ecDNA and Chromothripsis Regions by Sample', pad=20, fontsize=14)
    ax.tick_params(axis='y', labelsize=10)  # Larger y-axis tick labels
    
    # Create legend with larger font
    handles = [
        plt.Rectangle((0,0),1,1, color='#87CEEB', alpha=0.9),
        plt.Rectangle((0,0),1,1, color='#FF6B6B', alpha=0.9),
        plt.Rectangle((0,0),1,1, color='#6A4C93', alpha=0.9)
    ]
    labels = ['Chromothripsis', 'ecDNA + Chromothripsis overlap', 'ecDNA without Chromothripsis']
    ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
    
    # Add summary statistics with larger font
    stats_text = (
        f"Total samples: {len(data)}\n"
        f"With overlap: {(data['overlap'] > 0).sum()}\n"
        f"ecDNA only: {(data['category'] == 'ecDNA, no chromothripsis').sum()}\n"
        f"Average ecDNA size: {abs(data['length_ecDNA']).mean():.1f} Mb"
    )
    
    # Move stats box to better position for publication
    plt.figtext(0.7, 0.8, stats_text, 
           fontsize=8,
           bbox=dict(facecolor='white', edgecolor='none', alpha=0.8))
    
    return fig, ax

result = pd.read_csv("~/iCloud/dev/Sherlock-Lung/results2/chromothripsis-regions/ecDNA-chromothripsis/ecDNA-chromothripsis-summary.tsv", sep="\t")
# Create plot
fig, ax = create_genomic_barplot(result)
plt.show()

#save figure as pdf 
#fig.savefig("/Users/khandekara2/iCloud/dev/Sherlock-Lung/figures/ecDNA-chromothripsis-regions.pdf", bbox_inches='tight', dpi=300)

In [ ]:
result.head()

In [ ]:
result.Annotation.value_counts()